# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'SAADQN',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'acififlo',
 'close': None,
 'closeTime': None,
 'closeSource': 'iifoalfc',
 'high': 468.4,
 'highTime': 1623117888073,
 'highSource': 'yle nr1adciumietp5  dee',
 'low': 469.22,
 'lowTime': 1610214237034,
 'lowSource': ' eeinmucpeea5 yd ditlr1',
 'latestPrice': 481.2,
 'latestSource': 'IEX real time price',
 'latestTime': '3:35:55 PM',
 'latestUpdate': 1644906244524,
 'latestVolume': 47240039,
 'iexRealtimePrice': 480.62,
 'iexRealtimeSize': 102,
 'iexLastUpdated': 1628274236342,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 452.81,
 'previousVolume': 41883466,
 'change': 8.69,
 'changePercent': 0.01977,
 'volume': 45833327,
 'iexMarketPerce

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

35.5

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,101.410,46.27,N/A
1,AAL,13.835,-1.63,N/A
2,AAP,156.160,29.49,N/A
3,AAPL,474.390,36.44,N/A
4,ABBV,97.690,20.88,N/A
...,...,...,...,...
500,YUM,93.315,27.35,N/A
501,ZBH,142.620,713.49,N/A
502,ZBRA,295.560,32.46,N/A
503,ZION,35.517,13.05,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,63.630,2427.6,N/A
1,FIS,151.630,1572.15,N/A
2,GPC,98.878,969.34,N/A
3,ZBH,142.620,713.49,N/A
4,STZ,178.070,605.72,N/A
5,HLT,88.076,592.7,N/A
6,HRB,14.905,546.69,N/A
7,GLW,32.700,253.45,N/A
8,IQV,168.140,241.23,N/A
9,WMB,22.240,215.3,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,63.630,2427.6,308
1,FIS,151.630,1572.15,129
2,GPC,98.878,969.34,198
3,ZBH,142.620,713.49,137
4,STZ,178.070,605.72,110
5,HLT,88.076,592.7,222
6,HRB,14.905,546.69,1315
7,GLW,32.700,253.45,599
8,IQV,168.140,241.23,116
9,WMB,22.240,215.3,881


## Building a Better (and More Realistic) Momentum Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['enterpriseValue']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        pe_ratio = data[symbol]['quote']['peRatio']
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['ebitda']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
                                        pd.Series([
                                                    symbol, 
                                                    data[symbol]['quote']['latestPrice'], 
                                                    'N/A', 
                                                    pe_ratio,
                                                    'N/A',
                                                    pb_ratio,
                                                    'N/A',
                                                    ev_to_ebitda,
                                                    'N/A',
                                                    ev_to_gross_profit,
                                                    'N/A',
                                                    'N/A'
                                                   ], 
                                                  index = rv_columns), 
                                        ignore_index = True)
        

rv_dataframe.set_index('Ticker', inplace = True)
rv_dataframe

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
A,102.560,N/A,46.92,N/A,6.395986,N/A,1.0,N/A,11.366536,N/A,N/A
AAL,13.789,N/A,-1.7,N/A,-60.984491,N/A,1.0,N/A,3.205668,N/A,N/A
AAP,161.270,N/A,28.63,N/A,3.049885,N/A,1.0,N/A,3.163312,N/A,N/A
AAPL,461.230,N/A,34.95,N/A,22.692597,N/A,1.0,N/A,19.239035,N/A,N/A
ABBV,98.590,N/A,20.73,N/A,-21.293275,N/A,1.0,N/A,7.600621,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
YUM,94.100,N/A,27.94,N/A,-3.634270,N/A,1.0,N/A,14.019057,N/A,N/A
ZBH,142.235,N/A,701.9,N/A,2.367012,N/A,1.0,N/A,7.158158,N/A,N/A
ZBRA,289.220,N/A,31.17,N/A,8.651530,N/A,1.0,N/A,8.556800,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
AFL,37.820,N/A,9.71,N/A,0.950629,N/A,1.0,N/A,NaN,N/A,N/A
AIG,30.650,N/A,-5.8,N/A,0.401456,N/A,1.0,N/A,NaN,N/A,N/A
AIZ,127.800,N/A,20.25,N/A,1.321390,N/A,1.0,N/A,NaN,N/A,N/A
ALL,99.520,N/A,7.4,N/A,1.199250,N/A,1.0,N/A,NaN,N/A,N/A
ANTM,283.470,N/A,12.57,N/A,2.252930,N/A,1.0,N/A,NaN,N/A,N/A
AON,199.320,N/A,26.27,N/A,13.576894,N/A,1.0,N/A,NaN,N/A,N/A
BAC,27.033,N/A,12.8,N/A,0.888707,N/A,1.0,N/A,NaN,N/A,N/A
BK,38.859,N/A,8.4,N/A,0.814362,N/A,1.0,N/A,NaN,N/A,N/A
BKNG,1834.910,N/A,30.78,N/A,12.738583,N/A,1.0,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [17]:
for column in ['Price-to-Earnings Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [20]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics:
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

Ticker
A        46.92
AAL      -1.70
AAP      28.63
AAPL     34.95
ABBV     20.73
         ...  
YUM      27.94
ZBH     701.90
ZBRA     31.17
ZION     13.50
ZTS      47.96
Name: Price-to-Earnings Ratio, Length: 505, dtype: float64
Ticker
A        6.395986
AAL    -60.984491
AAP      3.049885
AAPL    22.692597
ABBV   -21.293275
          ...    
YUM     -3.634270
ZBH      2.367012
ZBRA     8.651530
ZION     0.778138
ZTS     29.243437
Name: Price-to-Book Ratio, Length: 505, dtype: float64
Ticker
A       1.0
AAL     1.0
AAP     1.0
AAPL    1.0
ABBV    1.0
       ... 
YUM     1.0
ZBH     1.0
ZBRA    1.0
ZION    1.0
ZTS     1.0
Name: EV/EBITDA, Length: 505, dtype: float64
Ticker
A       11.366536
AAL      3.205668
AAP      3.163312
AAPL    19.239035
ABBV     7.600621
          ...    
YUM     14.019057
ZBH      7.158158
ZBRA     8.556800
ZION    12.216590
ZTS     19.785205
Name: EV/GP, Length: 505, dtype: float64


,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
A,102.560,N/A,46.92,0.843564,6.395986,0.738614,1.0,0.50099,11.366536,0.556436,N/A
AAL,13.789,N/A,-1.70,0.114851,-60.984491,0.0158416,1.0,0.50099,3.205668,0.0673267,N/A
AAP,161.270,N/A,28.63,0.617822,3.049885,0.50099,1.0,0.50099,3.163312,0.0653465,N/A
AAPL,461.230,N/A,34.95,0.736634,22.692597,0.942574,1.0,0.50099,19.239035,0.855446,N/A
ABBV,98.590,N/A,20.73,0.419802,-21.293275,0.0257426,1.0,0.50099,7.600621,0.332673,N/A
...,...,...,...,...,...,...,...,...,...,...,...
YUM,94.100,N/A,27.94,0.592079,-3.634270,0.049505,1.0,0.50099,14.019057,0.748515,N/A
ZBH,142.235,N/A,701.90,0.994059,2.367012,0.407921,1.0,0.50099,7.158158,0.308911,N/A
ZBRA,289.220,N/A,31.17,0.681188,8.651530,0.813861,1.0,0.50099,8.556800,0.40198,N/A
